In [ ]:
# Loading required packages and libraries
import pandas as pd
import matplotlib.pyplot as plt
import warnings

In [ ]:
# Setting default options
pd.set_option("mode.copy_on_write", True)
warnings.simplefilter(action="ignore", category=FutureWarning)

In [4]:
# Loading required  data
df = pd.read_csv("../01_Data/01_Raw/nhis.csv.zip", compression="zip")
df.sample(5)

/var/folders/cj/jkp7xyld4_q3x3rk7xdx7jwh0000gn/T/ipykernel_34035/493222604.py:2: DtypeWarning: Columns (4,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../01_Data/01_Raw/nhis.csv.zip", compression="zip")


,YEAR,SERIAL,STRATA,PSU,NHISHID,HHWEIGHT,REGION,PERNUM,NHISPID,HHX,...,SMKECIGRND,CIGAREV,CIGARDAYSMO,CIGAR50LIFE,CIGARFREQNOW,PIPEV,PIPEFREQNOW,NONCIGEV,NONCIGNOW,TOBLSYR
683980,2007,12548,6007,2,2007020326,2730.0,1,2,20070203260102,20326,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1089294,2012,1582,6121,1,2012002328,1244.0,3,1,20120023280101,2328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1714802,2018,15672,7132,8,2018018690,3123.0,3,2,20180186900102,18690,...,0.0,0.0,96.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN
482550,2005,567,5037,1,2005000991,1733.0,3,1,20050009910101,991,...,NaN,1.0,96.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN
1656105,2017,27462,7126,33,2017033082,3281.0,3,1,20170330820101,33082,...,0.0,0.0,96.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN
